In [2]:
import os
import pickle

import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from tensorflow.keras.utils import load_img

from PIL import Image
from tqdm import tqdm

pd.options.mode.chained_assignment = None  # default='warn'
PATH_DF = "../dataset/1.json"
PATH = "../dataset/1/"
PATH_BLUR_IMAGE = "../dataset/2/"
PATH_FEATURES = "../dataset/feature.pkl"

In [ ]:
def  try_convert_val(value):
     try:
         return value["jibo"]
     except:
         return np.nan

In [12]:
df = pd.read_csv("/home/yuxiao/nii_project/dataset/df/feature_data.csv")

In [13]:
df = pd.read_json("/home/yuxiao/nii_project/dataset/1.json")
df["char"] = df["data"].apply(lambda x: x["char"])
df["url"] = df["data"].apply(lambda x: "" + x["url"][28:])
df["image_file"] = df["url"].apply(lambda x: x[10:])
df["jibo"] = df["data"].apply(lambda x: try_convert_val(x))
df = df[["char", "url", "image_file", "jibo"]]

In [40]:
path = r"../dataset/1"
# this list holds all the image filename
chars = []

# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
    # loops through each file in the directory
    chars.extend(
        file.name
        for file in files
        if file.name.endswith(".jpg")
        and file.name in df["image_file"].values
    )

In [35]:
for image_file in tqdm(df.image_file):
    img = cv2.imread(PATH + image_file)
    blue_channel, green_channel, red_channel = cv2.split(img)
    red_channel_removed = red_channel * 0
    img = cv2.merge((blue_channel, green_channel, red_channel_removed))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, dst1 = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)
    blur = cv2.blur(dst1, (2, 2))
    cv2.imwrite(PATH_BLUR_IMAGE + image_file, blur)

100%|██████████| 36869/36869 [09:51<00:00, 62.32it/s]


In [36]:
# load the model first and pass as an argument
model = VGG16()
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)


def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224, 224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img)
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1, 224, 224, 3)
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True, verbose=0)
    return features

2023-08-16 00:29:49.577481: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [42]:
data = {}
p = r"../dataset/2/char_features.pkl"

# lop through each image in the dataset
for char in tqdm(chars):
    # try to extract the features and update the dictionary
    try:
        feat = extract_features("../dataset/2/" + char, model)
        data[char] = feat
    # if something fails, save the extracted features as a pickle file (optional)
    except:
        with open(p, "wb") as file:
            pickle.dump(data, file)

# get a list of the filenames
filenames = np.array(list(data.keys()))

100%|██████████| 36869/36869 [1:27:30<00:00,  7.02it/s]


In [43]:
with open("../dataset/feature_2.pkl", "wb") as fp:
    pickle.dump(data, fp)
    print("dictionary saved successfully to file")

dictionary saved successfully to file
